# Instalacja bibliotek

Rozpoczniemy od instalacji biblioteki Logpai/Loglizer ze źródeł (Github).

Potem będzemy mogli używać jej, jak każdej innej biblioteki.

In [3]:
!git clone https://github.com/logpai/loglizer.git #nowa wersja nie ma Vextorize()

fatal: destination path 'loglizer' already exists and is not an empty directory.


In [4]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.0 MB/s eta 0:00:00


# Import bibliotek

In [5]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from graphviz import Graph

from sklearn.metrics import classification_report
krs = tf.keras

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
# Dodajemy ściągniętą bibliotekę, by była widoczna dla interpretera Pythona
sys.path.append("/content/loglizer")

Z biblioteki Loglizer będziemy potrzebować modułów do ładownia i preprocessingu.

In [18]:
from loglizer import dataloader, preprocessing

# Zadanie

Twoim zadaniem będzie skorzystanie z biblioteki loglizer służącej do analizy logów. Biblioteka posiada wbudowane mechanizmy do zamiany logów systemowych na postać, zdatną do przetwarzania przez rekurencyjne sieci neuronowe jak LSTM czy GRU.

 Na podstawie przygotowanych danych w postaci okien czasowych o szerokości K
(a więc `t-k, … t-2, t-1`) opisujących poprzednie logi i następny log w chwili t, musisz stworzyć sieć neuronową, oceniającą obecność anomalii.

W zależności od przyjętego podejścia (patrz niżej), albo będzie to sieć:
1. Przewidująca kolejny element sekwencji
2. Oceniające prawdopodobieństwo anomalii na podstawie poprzedniej sekwencji i kolejnego elementu.


Etykiety opisują, czy dane zdarzenie jest anomalią, czy też nie. Anomalia w tym kontekście, to zdarzenie „poza kolejnością”, coś co nie powinno się wydarzyć w czasie t, jeśli w poprzednim oknie zaszły określone zdarzenia.

___________

>           Przykład
            Dana jest sekwencja
          	    ABCDEF
          	Przyjmujemy okno czasowe = 3 i budujemy wycinki
          	    (ABC) -> D, anomalia = NIE
          	    (BCD) -> E, anomalia = NIE
          	    (CDE) -> F, anomalia = NIE
          	To są oczekiwane zdarzenia i względem takich zdarzeń będziesz uczyć algorytm.
          	Jeśli wystąpi zdarzenie
          	(ABC) -> X, to zdecydowanie jest to anomalia, bo spodziewalibyśmy się litery „D”.

___________________
# Szkolenie modelu

Twoim zdaniem będzie wyszkolić model rekurencyjny (LSTM albo GRU), który:
1. albo przewiduje NASTĘPNE zdarzenie, podobnie jak w przypadku następnego słowa albo znaku. Jest to zatem problem przewidywania sekwencji. Jeśli zdarzenie które naprawdę wystąpiło nie mieści się w top x (np. top 3, top 5) najbardziej prawdopodobnych alternatyw - jest anomalią.
2. albo dokonuje klasyfikacji binarnej na podstawie dwóch wejść: poprzedniej sekwencji zdarzeń oraz zdarzenia następnego. Model odpowiada zatem na pytanie: czy element jest czy nie jest anomalią.


# Ocena modelu

Ocena modelu na danych testowych będzie jednak wyglądała inaczej.
Istnieją co najmniej dwa podejścia i każde z nich warto przetestować.
Każde z tych podejść da Ci listę “anomalii”, które porównujesz z oczekiwanymi etykietami anomalii w zbiorze testowym - tak samo, jak robi się to zazwyczaj w eksperymentach.

1. **Podejście oparte o top K najbardziej prawdopodonych zdarzeń.**

    Dla każdej sekwencji testowej model rekurencyjny zwróci prawdopodobieństwa wystąpienia następnych elementów.
Dane testowe zawierają etykietę, opisującą jaki naprawdę wystąpił kolejny element.
Sprawdzenie na zasadzie TOP X, oznacza, że musisz zweryfikować, czy prawdopodobieństwo wystąpienia rzeczywistego elementu, zwiera się w top X (np. top 3, top 5) przewidzianych prawdopodobieństw przez model.

> Przykład 1:  
	Dana jest sekwencja ABC. Kolejny element to D.  
	Predykcje sieci (top 3 wyróżniono):  
		**P(D | ABC) = 0.9**  
		**P(E | ABC) = 0.05**  
		**P(F | ABC) = 0.001**   
		P(G | ABC) = 0.0001  
		Model przewiduje, że następny element to D, z prawdopodobieństwem 0.9.   
	W takim układze, element który naprawdę wystąpił (D) NIE JEST anomalią.

_________________


> Przykład 2:  
	Dana jest sekwencja ABC. Kolejny element to G.  
	Predykcje sieci (top 3 wyróżniono):  
		**P(D | ABC) = 0.9**  
		**P(E | ABC) = 0.05**  
		**P(F | ABC) = 0.001**   
		P(G | ABC) = 0.0001  
		Model przewiduje, że następny element to D, z prawdopodobieństwem 0.9.   
	W takim układze, element który naprawdę wystąpił (G) JEST anomalią, bo prawdopodobieństwo jego wystąpienia znajduje się poza TOP 3.



2. **Podejście oparte o wykorzystanie sieci z dwoma wejściami**

W drugim podejściu model ma dwa wejścia: poprzednią sekwencję i następny element. Model dokonuje klasyfikacji, czy wejście jest czy nie jest anomalią. Jest to zatem klasyczny przykład klasyfikacji.

[![](https://mermaid.ink/img/pako:eNptkLtOwzAUhl_FOnOKBGMGJNr0wsAEE0mHQ3yausQXOY6itOqCxENUvE76Xrg2QZXAg2Wf79dn6z9AqTlBCpVFs2UvWaGYXw-50cbuiSuBrKH3jlS5wzWbTO7ZNCf5RpwLVbHbdcxPA5nlHdrGdT2rhlPjyMOIs1xh44aTUT2j-iZ65leeux_PPJDFX88Fxn0WIkv_wfPH8FnuSYkxsIhofHUZrqv_ZasAH_Ou352_yl8FJCDJShTcd3K4zApwW5JUQOqPnDbY1q6AQh19FFunn3tVQupsSwm0hqOjTKBvU0K6wbrxU-LCafsUew51J2BQvWo9Zo7fXx2ALQ?type=png)](https://mermaid.live/edit#pako:eNptkLtOwzAUhl_FOnOKBGMGJNr0wsAEE0mHQ3yausQXOY6itOqCxENUvE76Xrg2QZXAg2Wf79dn6z9AqTlBCpVFs2UvWaGYXw-50cbuiSuBrKH3jlS5wzWbTO7ZNCf5RpwLVbHbdcxPA5nlHdrGdT2rhlPjyMOIs1xh44aTUT2j-iZ65leeux_PPJDFX88Fxn0WIkv_wfPH8FnuSYkxsIhofHUZrqv_ZasAH_Ou352_yl8FJCDJShTcd3K4zApwW5JUQOqPnDbY1q6AQh19FFunn3tVQupsSwm0hqOjTKBvU0K6wbrxU-LCafsUew51J2BQvWo9Zo7fXx2ALQ)


> Przykład:
	Dla sekwencji testowej ABC, dane testowe informują, że kolejny element to E. I **jest to anomalia** (y=1).  
	Model przyjmuje dwa wejścia:   
		1. poprzednia sekwencja: ABC  
		2. kolejny element: E  
	Predykcja modelu to po prostu prawdopodobieństwo, że E jest anomalią, jak w zwykłej klasyfikacji.


# Wczytanie danych

Pierwszym krokiem jest wczytanie danych z logów w postaci surowej oraz etykiet anomalii.
1.	Plik z logami ściągniemy jako csv spod adresu: `https://drive.google.com/uc?id=1I33xeBLzkH_0V5gmRpwk7Gk4hzUdsCKw&export=download`
2.	Plik z etykietami anomalii ściągniemy spod adresu: `https://drive.google.com/uc?id=1RDiWjGw-Cz6a6CTeE0GPixdRcJ2EfopG&export=download` - oficjalnego repozytorium biblioteki

Pliki tym razem muszą być ściągnięte bezpośrednio poleceniem `wget` ponieważ biblioteka `loglizer` nie potrafi czytać danych pod adresami URL.


In [8]:
!wget -O log_data.csv https://drive.google.com/uc?id=1I33xeBLzkH_0V5gmRpwk7Gk4hzUdsCKw&export=download

--2023-08-27 13:54:35--  https://drive.google.com/uc?id=1I33xeBLzkH_0V5gmRpwk7Gk4hzUdsCKw
Resolving drive.google.com (drive.google.com)... 74.125.23.101, 74.125.23.139, 74.125.23.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ivk7gdtekhhuqgqourmll2rtr30rrmnj/1693144425000/17484230016793547768/*/1I33xeBLzkH_0V5gmRpwk7Gk4hzUdsCKw?uuid=52173f14-5f25-457e-a522-6f061477791c [following]
--2023-08-27 13:54:40--  https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ivk7gdtekhhuqgqourmll2rtr30rrmnj/1693144425000/17484230016793547768/*/1I33xeBLzkH_0V5gmRpwk7Gk4hzUdsCKw?uuid=52173f14-5f25-457e-a522-6f061477791c
Resolving doc-0o-30-docs.googleusercontent.com (doc-0o-30-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting to doc-0o-30-

In [9]:
!wget -O anomaly_label.csv https://drive.google.com/uc?id=1RDiWjGw-Cz6a6CTeE0GPixdRcJ2EfopG&export=download

--2023-08-27 13:54:41--  https://drive.google.com/uc?id=1RDiWjGw-Cz6a6CTeE0GPixdRcJ2EfopG
Resolving drive.google.com (drive.google.com)... 74.125.23.101, 74.125.23.139, 74.125.23.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t9qbo4kcr7069qd2610fkpk1bbd2nrli/1693144425000/17484230016793547768/*/1RDiWjGw-Cz6a6CTeE0GPixdRcJ2EfopG?uuid=da082615-3440-4a2c-bd70-c9b3ba56ff79 [following]
--2023-08-27 13:54:44--  https://doc-0g-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t9qbo4kcr7069qd2610fkpk1bbd2nrli/1693144425000/17484230016793547768/*/1RDiWjGw-Cz6a6CTeE0GPixdRcJ2EfopG?uuid=da082615-3440-4a2c-bd70-c9b3ba56ff79
Resolving doc-0g-30-docs.googleusercontent.com (doc-0g-30-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting to doc-0g-30-

In [10]:
struct_log = r'/content/log_data.csv' # Plik csv z logiem (ściągnięty)
label_file = r'/content/anomaly_label.csv' # Plik z etykietami anomalii (ściągnięty)

In [11]:
log_data = pd.read_csv(struct_log, sep=",")
anomaly_labels = pd.read_csv(label_file, sep=";")

Dane logów wyglądają następująco:

1.	Kolumna **LineId** to numer linii w surowym logu
2.	**Date** i **time** to data i czas w formacie numeryczym
3.	**PID** jest identyfikatorem procesu w systemie. Przez większość czasu będzie używany przez bibliotekę **loglizer** do identyfikacji sesji.
4.	**Level** to poziom logowania
5.	**Component** to element systemu, który zgłosił wiadomość.
6.	**Content** - treść loga. Fragment `blk-XXXXX` jednoznacznie identyfikuje sesję użytkownika. Względem tego parametru biblioteka pogrupuje sekwencje.
7.	**EventID** - wynik mapowania treści loga na kod alfanumeryczny – tego będzie używać sieć neuronowa jako odpowiednika „słowa” albo „litery w analizę tekstu”.
8.	**EventTemplate** - tekstowy odpowiednik kodu alfanumerycznego. Wzorzec sekwencji.


In [12]:
log_data.head(5)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating


Biblioteka Loglizer podzieli dane wg. sesji (wyciągniętych `blk-XXXXX`) budując sekwencje zdarzeń.
> Przykład:  
Seria logów należących do sesji blk-001, może się ułożyć w wydarzenia:  
```
Receiving block…
NameSystem allocate Block….
Packer Responder …
```  
Co będzie odpowiadało sekwencji „słów” dla sieci neuronowej:  
`Blk-001:  E5, E22, E11`


Dane podzielimy w proporcji 80% do treningu, 20% do testu.

Okno poprzednich wydarzeń będzie obejmowało 10 wiadomości (10 „poprzednich słów” albo „poprzednich liter”).


In [13]:
train_ratio = 0.2
window_size = 10
np.random.seed(123)
(x_train, window_y_train, y_train), (x_test, window_y_test, y_test) = dataloader.load_HDFS(struct_log, label_file=label_file, window='session', window_size=window_size, train_ratio=train_ratio, split_type='uniform')

====== Input data summary ======
Loading /content/log_data.csv
62 251
Slicing 1587 sessions, with window 10
Slicing done, 8406 windows generated
Slicing 6353 sessions, with window 10
Slicing done, 25807 windows generated
Train: 8406 windows (464/8406 anomaly), 7942/8406 normal
Test: 25807 windows (842/25807 anomaly), 24965/25807 normal


Możemy sprawdzić, jak wyglądają dane po przetworzeniu: dla każdej sesji utworzono następujące elementy:

1.	X_train/test – okno zdarzeń („liter”/”słów”) poprzedzających wydarzenie
2.	Window_y_train/test – kolejne zdarzenie („litera”/”słowo”) w kolejce
3.	Y_train – czy zdarzenie jest anomalią


In [14]:
# Okno zdarzeń treningowych
x_train.head(5)

,SessionId,EventSequence
0,0,"[E5, E5, E22, E5, E11, E11, E9, E9, E11, E9]"
1,0,"[E5, E22, E5, E11, E11, E9, E9, E11, E9, E26]"
2,0,"[E22, E5, E11, E11, E9, E9, E11, E9, E26, E26]"
3,0,"[E5, E11, E11, E9, E9, E11, E9, E26, E26, E26]"
4,0,"[E11, E11, E9, E9, E11, E9, E26, E26, E26, E2]"


In [15]:
# Kolejne zdarzenie w kolejce
window_y_train.head(5)

0     E26
1     E26
2     E26
3      E2
4    #Pad
Name: Label, dtype: object

In [16]:
# Czy zdarzenie jest anomalią?
y_train.head(5)

0    0
1    0
2    0
3    0
4    0
Name: SessionLabel, dtype: int64

In [21]:
#niestety w najnowszej wersji logizera na gicie preproc ma tylko jedną klasę i nie jest to ta, której potrzebujemy :<

class Vectorizer(object):

    def fit_transform(self, x_train, window_y_train, y_train):
        self.label_mapping = {eid: idx for idx, eid in enumerate(window_y_train.unique(), 2)}
        self.label_mapping["#OOV"] = 0
        self.label_mapping["#Pad"] = 1
        self.num_labels = len(self.label_mapping)
        return self.transform(x_train, window_y_train, y_train)

    def transform(self, x, window_y, y):
        x["EventSequence"] = x["EventSequence"].map(lambda x: [self.label_mapping.get(item, 0) for item in x])
        window_y = window_y.map(lambda x: self.label_mapping.get(x, 0))
        y = y
        data_dict = {"SessionId": x["SessionId"].values, "window_y": window_y.values, "y": y.values, "x": np.array(x["EventSequence"].tolist())}
        return data_dict

Ostatnią rzeczą, której dokona za nas biblioteka, jest zamiana znaków kodowych logów na identyfikatory numeryczne, których użyjemy jako „słownika” w warstwie **Embedding**.

In [22]:

feature_extractor = Vectorizer()
train_dataset = feature_extractor.fit_transform(x_train, window_y_train, y_train)
test_dataset = feature_extractor.transform(x_test, window_y_test, y_test)


In [23]:
train_dataset.keys()

dict_keys(['SessionId', 'window_y', 'y', 'x'])

In [24]:
train_dataset['x'][:5]


array([[8, 8, 0, 8, 6, 6, 5, 5, 6, 5],
       [8, 0, 8, 6, 6, 5, 5, 6, 5, 2],
       [0, 8, 6, 6, 5, 5, 6, 5, 2, 2],
       [8, 6, 6, 5, 5, 6, 5, 2, 2, 2],
       [6, 6, 5, 5, 6, 5, 2, 2, 2, 3]])

In [25]:
x_train

,SessionId,EventSequence
0,0,"[8, 8, 0, 8, 6, 6, 5, 5, 6, 5]"
1,0,"[8, 0, 8, 6, 6, 5, 5, 6, 5, 2]"
2,0,"[0, 8, 6, 6, 5, 5, 6, 5, 2, 2]"
3,0,"[8, 6, 6, 5, 5, 6, 5, 2, 2, 2]"
4,0,"[6, 6, 5, 5, 6, 5, 2, 2, 2, 3]"
...,...,...
8401,1585,"[8, 6, 5, 6, 5, 6, 5, 2, 2, 2]"
8402,1586,"[8, 8, 8, 0, 6, 5, 6, 5, 2, 2]"
8403,1586,"[8, 8, 0, 6, 5, 6, 5, 2, 2, 6]"
8404,1586,"[8, 0, 6, 5, 6, 5, 2, 2, 6, 5]"


Utworzone zbiory danych train/test dataset zawierają te same informacje, co wcześniej ale już wyłącznie w postaci macierzowej i numerycznej.

Na przykład:

```
Sekwencja E1, E10, E5  

zostaje zamieniona na

1, 10, 5
```

In [26]:
# Kolejne zdarzenie (słowo/litera) w sekwencji
train_dataset['window_y'][:5]

array([2, 2, 2, 3, 1])

In [27]:
# Czy jest anomalią?
train_dataset['y'][:5]

array([0, 0, 0, 0, 0])

# Szkolenie sieci

Twoim zdaniem będzie przygotowanie dowolnej sieci, szkolonej na danych treningowych, wykorzystującej podane informacje.
Sieć powinna przewidywać wystąpienie anomalii.

_______________________

**PODPOWIEDŹ**
Warto wykorzystać następujące elementy:

1.	Sieć przewidująca następny element:
    1.	**feature_extractor.num_labels** zawiera „wielkość słownika”, czyli wszystkie dopuszczalne „litery”/”słowa”. Będzie to argumentem do dwóch elementów sieci:
        1.	Warstwy Embedding: `input_dim=feature_extractor.num_labels+1` – słownik dopuszczalnych embeddingów (+1 dla nieznanego słowa/nieznanego znaku) oraz `input_length=X_train.shape[1]` - dł. sekwencji testowej.
        2.	Warstwy wyjścia z aktywacją softmax: Dense(feature_extractor.num_labels+1, activation='softmax') do przewidywania następnego elementu.
    2.	Funkcja kosztu ` categorical_crossentropy` ponieważ przewidujemy wiele kategorii/możliwych wartości słów.
    3.	Podczas testu będziesz wybierać anomalie na podstawie metody TOP X albo odcięcia prawdopodobieństwa.
2.	Być może warto zastanowić się nad siecią z dwoma wejściami?
    1.	Jedno wejście – poprzednie zdarzenia (embedding o wejściu sekwencji  `input_length=X_train.shape[1]` oraz słowikiem jak wyżej)
    2.	Kolejne wejście – zdarzenie następne (embedding z jednym elementem `input_length=1` i słownikiem)
    3.	Wyjście – prawdopodobieństwo anomalii (wyjście: `sigmoid` funkcja kosztu: `binary_crossentropy`)


In [99]:
from tensorflow.keras.callbacks import EarlyStopping

model_nastepny_element = krs.Sequential([
    krs.layers.Embedding(input_dim=feature_extractor.num_labels+1, output_dim=12, input_length=train_dataset['x'].shape[1]),
    krs.layers.Bidirectional(krs.layers.LSTM(256, return_sequences=False)),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(256, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(128, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(64, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(16, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(8, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(4, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(2, activation='relu'),
    krs.layers.Dropout(0.2),
    krs.layers.Dense(feature_extractor.num_labels+1, activation='softmax')
])
model_nastepny_element.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=feature_extractor.num_labels+1)])
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [100]:
history_model_next_elem = model_nastepny_element.fit(train_dataset['x'],  krs.utils.to_categorical(train_dataset['window_y'], feature_extractor.num_labels+1), epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stop])

Epoch 1/20
211/211 [==============================] - 24s 84ms/step - loss: 2.4433 - f1_score: 0.0531 - val_loss: 2.3058 - val_f1_score: 0.0543
Epoch 2/20
211/211 [==============================] - 17s 83ms/step - loss: 2.2269 - f1_score: 0.0512 - val_loss: 2.0902 - val_f1_score: 0.0543
Epoch 3/20
211/211 [==============================] - 16s 76ms/step - loss: 2.0504 - f1_score: 0.0512 - val_loss: 1.9146 - val_f1_score: 0.0543
Epoch 4/20
211/211 [==============================] - 17s 79ms/step - loss: 1.9094 - f1_score: 0.0512 - val_loss: 1.7737 - val_f1_score: 0.0543
Epoch 5/20
211/211 [==============================] - 16s 78ms/step - loss: 1.7982 - f1_score: 0.0512 - val_loss: 1.6619 - val_f1_score: 0.0543
Epoch 6/20
211/211 [==============================] - 18s 85ms/step - loss: 1.7107 - f1_score: 0.0512 - val_loss: 1.5731 - val_f1_score: 0.0543
Epoch 7/20
211/211 [==============================] - 16s 76ms/step - loss: 1.6427 - f1_score: 0.0512 - val_loss: 1.5040 - val_f1_score:

In [101]:

model_nastepny_element.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, 10, 12)            156       
                                                                 
 bidirectional_21 (Bidirecti  (None, 512)              550912    
 onal)                                                           
                                                                 
 dropout_108 (Dropout)       (None, 512)               0         
                                                                 
 dense_109 (Dense)           (None, 256)               131328    
                                                                 
 dropout_109 (Dropout)       (None, 256)               0         
                                                                 
 dense_110 (Dense)           (None, 128)               32896     
                                                     

**Autotuner** - niestety nie zadziałał z nienzanych mi podowód

In [156]:
def build_model(hp):
    model = krs.models.Sequential()


    hp_units = hp.Int('units', min_value=4, max_value=128, step=16)
    hp_activations = hp.Choice('activation', values=['tanh', 'relu'])

    model.add(krs.layers.Embedding(input_dim=feature_extractor.num_labels+1, output_dim=12, input_length=train_dataset['x'].shape[1]))
    model.add(krs.layers.Bidirectional(krs.layers.LSTM(256, return_sequences=False)))
    model.add(krs.layers.Dense(int(hp_units), hp_activations))
    model.add(krs.layers.Dropout(0.2))
    model.add(krs.layers.Dense(int(hp_units), hp_activations))
    model.add(krs.layers.Dropout(0.2))
    model.add(krs.layers.Dense(int(hp_units), hp_activations))
    model.add(krs.layers.Dropout(0.2))
    model.add(krs.layers.Dense(13, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=feature_extractor.num_labels+1)])
    return model



In [121]:
%pip install keras_tuner
import kerastuner as kt

<ipython-input-121-5e1ab4bc8f80>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [157]:
tuner = kt.Hyperband(
    build_model,                    # Funkcja budująca moel
    objective= 'categorical_crossentropy',               # Funkcja celu
    max_epochs=10,                  # Max. liczba epok
    hyperband_iterations=2,         # ilość iteracji wyboru parametrów
    directory="tuner_results",      # katalog do zapisu wyników
    project_name='tuner',
    executions_per_trial=5)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='categorical_crossentropy', patience=5)


In [158]:
tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])


Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
52                |116               |units
relu              |relu              |activation
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/tuners/hyperband.py", line 425, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras_tuner/engine/hypermodel.py", line

RuntimeError: ignored

In [ ]:
best_activation = tuner.get_best_hyperparameters()[0].get('activation')
best_no_units = tuner.get_best_hyperparameters()[0].get('units')
print(f"Best activation: {best_activation}, best no. units: {best_no_units}")

In [ ]:
best_params = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_params)
history = model.fit(x_train, y_train, epochs=20, validation_split=0.2)

**Anomaly**


In [105]:
inp_seq = krs.layers.Input(shape=(train_dataset['x'].shape[1]))
embed = krs.layers.Embedding(input_dim=feature_extractor.num_labels+1, output_dim=12, input_length=train_dataset['x'].shape[1])(inp_seq)
lstm_seq = krs.layers.LSTM(512, return_sequences=False)(embed)
drop_seq = krs.layers.Dropout(0.2)(lstm_seq)
dense1_seq = krs.layers.Dense(256, activation='relu')(drop_seq)
drop2_seq = krs.layers.Dropout(0.2)(dense1_seq)
dense2_seq = krs.layers.Dense(128, activation='relu')(drop2_seq)
drop3_seq = krs.layers.Dropout(0.2)(dense2_seq)
dense4_seq = krs.layers.Dense(64, activation='relu')(drop3_seq)
drop4_seq = krs.layers.Dropout(0.2)(dense4_seq)
dense5_seq = krs.layers.Dense(16, activation='relu')(drop4_seq)
drop5_seq = krs.layers.Dropout(0.2)(dense5_seq)
dense6_seq = krs.layers.Dense(8, activation='relu')(drop5_seq)
drop6_seq = krs.layers.Dropout(0.2)(dense6_seq)
dense7_seq = krs.layers.Dense(4, activation='relu')(drop6_seq)
drop7_seq = krs.layers.Dropout(0.2)(dense7_seq)

inp_next_elem = krs.layers.Input(shape=(1,))
embed_next_elem = krs.layers.Embedding(input_dim=feature_extractor.num_labels+1, output_dim=12, input_length=1)(inp_next_elem)
dense_elem = krs.layers.Dense(feature_extractor.num_labels+1, activation='relu')(inp_next_elem)

concat = krs.layers.Concatenate(axis=1)([drop7_seq, dense_elem])
out = krs.layers.Dense(1, activation='sigmoid')(concat)
model_anomaly = krs.Model(inputs=[inp_seq, inp_next_elem], outputs=out)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_anomaly.compile(loss='binary_crossentropy', optimizer='adam', metrics=[krs.metrics.Precision(), krs.metrics.Recall()])

In [106]:
history_anomaly = model_anomaly.fit(
    [train_dataset['x'], train_dataset['window_y']], train_dataset['y'],
    epochs=30, batch_size=32, callbacks=[early_stop])

Epoch 1/30
263/263 [==============================] - ETA: 0s - loss: 0.3078 - precision_2: 0.4231 - recall_2: 0.0237

263/263 [==============================] - 46s 155ms/step - loss: 0.3078 - precision_2: 0.4231 - recall_2: 0.0237
Epoch 2/30
263/263 [==============================] - ETA: 0s - loss: 0.2249 - precision_2: 0.4815 - recall_2: 0.0841

263/263 [==============================] - 41s 156ms/step - loss: 0.2249 - precision_2: 0.4815 - recall_2: 0.0841
Epoch 3/30
263/263 [==============================] - ETA: 0s - loss: 0.1993 - precision_2: 0.5312 - recall_2: 0.0733

263/263 [==============================] - 41s 156ms/step - loss: 0.1993 - precision_2: 0.5312 - recall_2: 0.0733
Epoch 4/30
263/263 [==============================] - ETA: 0s - loss: 0.1913 - precision_2: 0.5833 - recall_2: 0.0754

263/263 [==============================] - 40s 153ms/step - loss: 0.1913 - precision_2: 0.5833 - recall_2: 0.0754
Epoch 5/30
263/263 [==============================] - ETA: 0s - loss: 0.1811 - precision_2: 0.5432 - recall_2: 0.0948

...

263/263 [==============================] - 43s 164ms/step - loss: 0.1635 - precision_2: 0.5714 - recall_2: 0.3276
Epoch 27/30
263/263 [==============================] - ETA: 0s - loss: 0.1631 - precision_2: 0.5804 - recall_2: 0.3190

263/263 [==============================] - 43s 162ms/step - loss: 0.1631 - precision_2: 0.5804 - recall_2: 0.3190
Epoch 28/30
263/263 [==============================] - ETA: 0s - loss: 0.1640 - precision_2: 0.5526 - recall_2: 0.3621

263/263 [==============================] - 42s 161ms/step - loss: 0.1640 - precision_2: 0.5526 - recall_2: 0.3621
Epoch 29/30
263/263 [==============================] - ETA: 0s - loss: 0.1639 - precision_2: 0.5627 - recall_2: 0.3772

263/263 [==============================] - 43s 164ms/step - loss: 0.1639 - precision_2: 0.5627 - recall_2: 0.3772
Epoch 30/30
263/263 [==============================] - ETA: 0s - loss: 0.1634 - precision_2: 0.5739 - recall_2: 0.3599

263/263 [==============================] - 42s 161ms/step - loss: 0.1634 - precision_2: 0.5739 - recall_2: 0.3599


# Przewidywanie anomalii

Sprawdź jakość predykcji Twojej sieci na zbiorze testowym.

1.	Jeśli Twoja sieć przewiduje następny element – do wykrywania anomalii możesz użyć metody TOP X (jeśli następny element nie jest w TOP X przewidzianych prawdopodobieństw to będzie anomalią) albo metody odcięcia (jeśli prawdopodobieństwo następnego elementu jest mniejsze niż X, to jest anomalią).
2.	Jeśli Twoja sieć przewiduje anomalie na podstawie dwóch wejść – możesz to traktować jak zwykłą klasyfikację.


## Sieć przewidująca następny element

Poniższy fragment pokazuje sposób rozwiązania z wykorzystaniem sieci przewidującej następny element.
* **Krok 1** - predykcja następnego elementu, mając daną sekwencję testową. Każda predykcja to wektor o długości takiej jak “słownik” wszystkich dopuszczalnych elementów .
* **Krok 2** - posortowanie predykcji, żeby wybrać top N (np. top 3) tych o najwyższym prawdopodobieństwie.
* **Krok 3** - sprawdzenie, czy zdarzenie, które nastąpiło mieści się w top N. Jeśli nie – zostaje uznane za anomalię.



In [102]:
yhat_next_elem = model_nastepny_element.predict(test_dataset['x'])

807/807 [==============================] - 18s 20ms/step


In [103]:
is_next_event_anomaly = []
for idx in range(test_dataset['window_y'].shape[0]):
    next_event = test_dataset['window_y'][idx]
    most_probably_events = yhat_next_elem[idx].argsort()[::-1]
    top = most_probably_events[:3]
    is_next_event_anomaly.append(next_event not in top)

In [104]:

print(classification_report(y_true=test_dataset['y'], y_pred=is_next_event_anomaly))

              precision    recall  f1-score   support

           0       0.97      0.44      0.61     24965
           1       0.03      0.60      0.07       842

    accuracy                           0.45     25807
   macro avg       0.50      0.52      0.34     25807
weighted avg       0.94      0.45      0.59     25807



## Model pracujący na dwóch wejściach

Poniższy fragment pokazuje sposób rozwiązania z wykorzystaniem sieci klasyfikującej anomalie. Jest to zwyczajna sieć do klasyfikacji binarnej. Odróżnia się tym, że posiada dwa wejścia: sekwencję poprzedzającą i zdarzenie, które nastąpiło potem.

In [107]:
yhat_anomaly = model_anomaly.predict([test_dataset['x'], test_dataset['window_y']])

807/807 [==============================] - 28s 34ms/step


In [108]:
yhat_anomaly_cls = yhat_anomaly > 0.5

In [109]:
print(classification_report(y_true=test_dataset['y'], y_pred=yhat_anomaly_cls))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     24965
           1       0.00      0.00      0.00       842

    accuracy                           0.97     25807
   macro avg       0.48      0.50      0.49     25807
weighted avg       0.94      0.97      0.95     25807



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
